In [2]:
import nltk
import ssl
from nltk.corpus import stopwords
import re
from nltk.corpus import reuters
import string
nltk.download('reuters')
nltk.download('punkt')

[nltk_data] Downloading package reuters to
[nltk_data]     /Users/edwardbaker/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/edwardbaker/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Extract fileids from the reuters corpus
fileids = reuters.fileids()
# Initialize empty lists to store categories and raw text
categories = []
text = []
# Loop through each file id and collect each files categories and raw text
for file in fileids:
    categories.append(reuters.categories(file))
    text.append(reuters.raw(file))

# split into sentences for easier cleaning for bigram
for i in range(len(text)):
    text[i] = nltk.sent_tokenize(text[i])
    for j in range(len(text[i])):
        text[i][j] = text[i][j].replace("\n", "")
# Now text is a list in which each element is a sentence collected from the Reuters news.

In [4]:
# a Remove any URLs and numbers from each sentence
for i in range(len(text)):
    for j in range(len(text[i])):
        text[i][j] = re.sub(r'https?\S+', '',   text[i][j])
        text[i][j] = re.sub(r'\d+', '',   text[i][j]) 

In [5]:
# b Lowercase each sentence, tokenize it using the NLTK tokenizer.
for i in range(len(text)):
    for j in range(len(text[i])):
        text[i][j] = nltk.word_tokenize(text[i][j].lower())

In [6]:
# c Now remove all stopwords, punctuations in each documents
# consider punctuations in string.punct-uation in python and stem all tokens.
stop_words = stopwords.words('english')
for i in range(len(text)):
    for j in range(len(text[i])):
        text[i][j] = [word for word in text[i][j] if word not in stop_words and word not in string.punctuation]
        for k in range(len(text[i][j])):
            text[i][j][k] = nltk.PorterStemmer().stem(text[i][j][k])


In [7]:
# pad with start and end tokens
for i in range(len(text)):
    for j in range(len(text[i])):
        text[i][j].insert(0,"start_token")
        text[i][j].append("end_token")

In [8]:
print(text[1])

[['start_token', 'china', 'daili', 'say', 'vermin', 'eat', 'pct', 'grain', 'stock', 'survey', 'provinc', 'seven', 'citi', 'show', 'vermin', 'consum', 'seven', 'pct', 'china', "'s", 'grain', 'stock', 'china', 'daili', 'said', 'end_token'], ['start_token', 'also', 'said', 'year', 'mln', 'tonn', 'pct', 'china', "'s", 'fruit', 'output', 'left', 'rot', 'mln', 'tonn', 'pct', 'veget', 'end_token'], ['start_token', 'paper', 'blame', 'wast', 'inadequ', 'storag', 'bad', 'preserv', 'method', 'end_token'], ['start_token', 'said', 'govern', 'launch', 'nation', 'programm', 'reduc', 'wast', 'call', 'improv', 'technolog', 'storag', 'preserv', 'greater', 'product', 'addit', 'end_token'], ['start_token', 'paper', 'gave', 'detail', 'end_token']]


In [9]:
# d creating the bigram model
lookup = {}

# go through every article
for article in text:
    # and every sentence in every article
    for sent in article:
        for i in range(len(sent)-1):
            if sent[i] not in lookup:
                lookup[sent[i]] = {}
            try:
                lookup[sent[i]][sent[i+1]] += 1
            except:
                lookup[sent[i]][sent[i+1]] = 1 

In [10]:
# e convert counts to probabilities
for tokens in lookup:
    count =  sum(lookup[tokens].values())
    for key in lookup[tokens]:
        lookup[tokens][key] = lookup[tokens][key] / count

In [11]:
# f sentence generation
import random
def generate_sentence():
    sentence = "start_token"
    word = "start_token"
    while word != "end_token":
        # word is a 1 elt array containing the selected value from the prob distribution
        word = random.choices(list(lookup[word].keys()), weights=lookup[word].values(), k=1)
        word = word[0]
        sentence = sentence + " " + word
    return sentence

In [17]:
print("Sentence 1: " + generate_sentence())
print("Sentence 2: " + generate_sentence())
print("Sentence 3: " + generate_sentence())

Sentence 1: start_token sever compani also underwritten lt milex corp say exchang commiss nation seri six-month deposit import polici action countri agre mutual omaha bob dole also rate commerci bank portug tax incent bonus tonn sugar institut west texa refineri crude oil market lead '' end_token
Sentence 2: start_token forecast current oil industri product stood semiconductor trade sanction japan end_token
Sentence 3: start_token financ cost reduc event past eight pct said may record cotton govern help motiv end_token
